In [1]:
import numpy as np
import cv2
import skimage
from skimage.io import imread
from skimage.color import rgb2lab, lab2rgb
from skimage import color

from sklearn.cluster import KMeans
import math

In [2]:
def is_darker(color_space, centroid1, centroid2):

    if color_space == "HSV":
        # Use Value
        return centroid1[2] > centroid2[2]
    elif color_space == "RGB":
        # Calculate the average intensity of the two centroids
        intensity1 = (centroid1[0] + centroid1[1] + centroid1[2]) / 3.0
        intensity2 = (centroid2[0] + centroid2[1] + centroid2[2]) / 3.0
        return intensity1 > intensity2
    else:
        # Use the L* (luminance) component in the L*a*b* color space (index 0)
        return centroid1[0] > centroid2[0]

In [7]:
# Each channel
def extract_skin_pixels(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, color_space_mappings[color_space])
    #red_channel = img[:, :, 0]
    color_df = pd.DataFrame(img.reshape(-1, 3), columns=list(color_space))
    
    kmeans = KMeans(n_clusters=2, n_init="auto", random_state=42)
    kmeans.fit(color_df)
    color_df["Cluster"] = kmeans.labels_
    centroids = kmeans.cluster_centers_
    
    # Create a mask for the darker cluster, 0 is darker
    darker_cluster = 0 if is_darker(color_space, centroids[0], centroids[1]) else 1
    color_df["Cluster"] == darker_cluster
    
    return list(color_df[color_df["Cluster"] == 1][c_channel])

In [1]:
def mean_skin_color(image_path):
    
    color_space = "RGB"
    color_attributes = list(color_space)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    color_df = pd.DataFrame(img.reshape(-1, 3), columns=color_attributes)
    kmeans = KMeans(n_clusters=2, n_init="auto", random_state=42)
    kmeans.fit(color_df)
    color_df["Cluster"] = kmeans.labels_
    centroids = kmeans.cluster_centers_
    darker_cluster = 0 if is_darker(color_space, centroids[0], centroids[1]) else 1
    mask_darker = color_df["Cluster"] == darker_cluster
    mask_darker = mask_darker.values.reshape(img.shape[0], img.shape[1])
    mask_darker = mask_darker.astype(np.uint8)
    skin_color = cv2.mean(img, mask=mask_darker)

    return skin_color

In [4]:
def ITA(image):
    """
    Calculates the individual typology angle (ITA) for a given
    RGB image.

    Inputs:
        image - (str) RGB image file path

    Outputs:
        ITA - (float) individual typology angle
    """

    # Convert to CIE-LAB color space
    RGB = Image.open(image)
    CIELAB = np.array(color.rgb2lab(RGB))

    # Get L and B (subset to +- 1 std from mean)
    L = CIELAB[:, :, 0]
    L = np.where(L != 0, L, np.nan)
    std, mean = np.nanstd(L), np.nanmean(L)
    L = np.where(L >= mean - std, L, np.nan)
    L = np.where(L <= mean + std, L, np.nan)
    
    B = CIELAB[:, :, 2]
    B = np.where(B != 0, B, np.nan)
    std, mean = np.nanstd(B), np.nanmean(B)
    B = np.where(B >= mean - std, B, np.nan)
    B = np.where(B <= mean + std, B, np.nan)

    # Calculate ITA
    ITA = math.atan2(np.nanmean(L) - 50, np.nanmean(B)) * (180 / np.pi)

    return ITA

In [5]:
def ITA_label(ITA, method):
    """
    Maps an input ITA to a fitzpatrick label given
    a choice method

    Inputs:
        ITA - (float) individual typology angle
        method - (str) 'kinyanjui' or None

    OutputsL
        (int) fitzpatrick type 1-6
    """

    # Use thresholds from kinyanjui et. al.
    if method == 'kinyanjui':
        if ITA > 55:
            return 1
        elif ITA > 41:
            return 2
        elif ITA > 28:
            return 3
        elif ITA > 19:
            return 4
        elif ITA > 10:
            return 5
        elif ITA <= 10:
            return 6
        else:
            return None

    # Use empirical thresholds
    else:
        if ITA >= 45:
            return 1
        elif ITA > 28:
            return 2
        elif ITA > 17:
            return 3
        elif ITA > 5:
            return 4
        elif ITA > -20:
            return 5
        elif ITA <= -20:
            return 6
        else:
            return None